In [57]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [44]:
reform = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  }
}, country_id="us")

reform_taxable = Reform.from_dict({
  "gov.contrib.states.or.rebate.state_tax_exempt": {
    "2024-01-01.2100-12-31": True
  },
  "gov.contrib.ubi_center.basic_income.amount.person.flat": {
    "2025-01-01.2025-12-31": 1160,
    "2026-01-01.2026-12-31": 1605,
    "2027-01-01.2027-12-31": 1686
  },
  "gov.contrib.ubi_center.basic_income.taxable": {
    "2024-01-01.2100-12-31": True
  }
}, country_id="us")

In [45]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
reformed_taxable = Microsimulation(reform=reform_taxable)


In [64]:
def calculate_poverty_impact(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    
    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)
    
    baseline_poverty_oregon = baseline_poverty[state_codes == "OR"].mean()
    reform_poverty_oregon = reform_poverty[state_codes == "OR"].mean()
    
    # Calculate relative poverty reduction
    relative_poverty_reduction = (reform_poverty_oregon - baseline_poverty_oregon) / baseline_poverty_oregon
    
    return baseline_poverty_oregon, relative_poverty_reduction


In [65]:
# Initialize an empty DataFrame to store all results
all_results = pd.DataFrame(columns=["year", "baseline_poverty", "relative_poverty_reduction", "relative_poverty_reduction_taxable"])


In [66]:
# Calculate for each year and add to the DataFrame
for year in range(2025, 2028):
    baseline_poverty, relative_poverty_reduction = calculate_poverty_impact(baseline, reformed, year)
    _, relative_poverty_reduction_taxable = calculate_poverty_impact(baseline, reformed_taxable, year)
    
    # Create a new row of data
    new_row = pd.DataFrame({
        "year": [year],
        "baseline_poverty": [baseline_poverty],
        "relative_poverty_reduction": [relative_poverty_reduction],
        "relative_poverty_reduction_taxable": [relative_poverty_reduction_taxable]
    })
    
    # Append the new row to all_results
    all_results = pd.concat([all_results, new_row], ignore_index=True)


In [67]:
# Display the results
print(all_results)

   year  baseline_poverty  relative_poverty_reduction  \
0  2025          0.106336                   -0.222731   
1  2026          0.106623                   -0.277636   
2  2027          0.104942                   -0.271545   

   relative_poverty_reduction_taxable  
0                           -0.222731  
1                           -0.249429  
2                           -0.251366  


In [69]:
# Define colors
colors = {
    "not_taxable": "#4d94ff",  # Light blue
    "taxable": "#999999",      # Light grey
}

# Create the plot
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=all_results['year'], 
    y=all_results['relative_poverty_reduction'],
    mode='lines',
    name='Rebate not taxable',
    line=dict(color=colors['not_taxable'], width=2)
))

fig.add_trace(go.Scatter(
    x=all_results['year'], 
    y=all_results['relative_poverty_reduction_taxable'],
    mode='lines',
    name='Rebate taxable under the federal income tax program',
    line=dict(color=colors['taxable'], width=2)
))

# Update layout
fig.update_layout(
    title='Oregon Rebate Impact on Poverty Over Time',
    xaxis_title='Year',
    yaxis_title='Relative Poverty Reduction (%)',
    legend_title='Reform Type',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    ),
    height=600,
    width=800,
)

# Update x-axis to show only three steps
min_year = all_results['year'].min()
max_year = all_results['year'].max()
mid_year = min_year + (max_year - min_year) // 2

fig.update_xaxes(
    tickvals=[min_year, mid_year, max_year],
    ticktext=[str(min_year), str(mid_year), str(max_year)],
    tickformat='d'
)

# Update y-axis
fig.update_yaxes(tickformat='.0%')

# Update hover template
fig.update_traces(
    hovertemplate='Year: %{x}<br>Relative Poverty Reduction: %{y:.2%}<extra></extra>'
)

fig = format_fig(fig)
fig.show()